In [400]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from datetime import datetime

In [401]:
from main import main

main()

Hello from automate-minimas!


In [402]:
def fetch_data(ticker, start_date, end_date):
    """Fetch historical data for a given ticker and date range using yfinance."""
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        raise ValueError(f"No data found for ticker {ticker} in the given date range.")
    return data

In [403]:
def find_extrema(data, column, peak_count=2, maximum=True):
    """Find specified number of minimas or maximas in the given column of the data."""
    if column not in data.columns:
        raise ValueError(f"Column {column} not found in data.")
    # Ensure the column data is a 1-D array
    column_data = data[column].dropna().values.squeeze()

    # Find extrema
    if not maximum:
        column_data = -column_data
    price_range = column_data.max() - column_data.min()
    prominence = price_range * 0.4  # Adjust prominence as needed
    extrema_indices, _ = find_peaks(column_data, prominence=prominence)
    # extrema_indices =
    extrema = data.iloc[extrema_indices]
    # .nlargest(peak_count, column)

    return extrema

In [404]:
def display_results(extrema, column):
    """Display the extrema results in a clear format."""
    print(f"\nDetected extrema for column: {column}")
    for idx, row in extrema.iterrows():
        print(f"Date: {idx}, Value: {row[column]}")

In [405]:
low_minima.iloc[:1]

Price,Close,High,Low,Open,Volume
Date,,,,,
2025-08-21,113.790001,114.75,113.330002,114.459999,284300


In [406]:
# Example usage


ticker = "CPA"
start_date = "2025-08-12"
end_date = "2025-09-10"
# ticker = "agco"
# start_date = "2025-07-30"
# end_date = "2025-09-03"
# ticker = "BLK"
# start_date = "2025-08-12"
# end_date = "2025-09-04"

# Fetch data
data = fetch_data(ticker, start_date, end_date)
data.columns = data.columns.droplevel(1)  # need to drop company name level

# Find extrema for 'Low' and 'High' columns
low_minima = find_extrema(data, "Low", maximum=False)
high_maxima = find_extrema(data, "High", maximum=True)


def ensure_maximum_after_minimum(low_minima, high_maxima):
    """Returns first minimum followed by the first maximum after it."""
    minimum = low_minima.iloc[[0]]  # purposefuly keeping dataframe format
    maximum = high_maxima[high_maxima.index > minimum.index].iloc[[0]]
    return minimum, maximum


low_minima, high_maxima = ensure_maximum_after_minimum(low_minima, high_maxima)
# Display results
print("Low MIniamas:")
display_results(low_minima, "Low")
print("\nHigh Maximas:")
display_results(high_maxima, "High")


C:\Users\Maciej\AppData\Local\Temp\ipykernel_18480\6992924.py:3: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Low MIniamas:

Detected extrema for column: Low
Date: 2025-08-21 00:00:00, Value: 113.33000183105469

High Maximas:

Detected extrema for column: High
Date: 2025-08-28 00:00:00, Value: 120.2699966430664


In [407]:
import plotly.graph_objects as go


def plot_candlestick_with_extrema(data, low_minima, high_maxima):
    """Plot a candlestick chart with extrema points using Plotly."""
    fig = go.Figure()
    # Candlestick
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data["Open"],
            high=data["High"],
            low=data["Low"],
            close=data["Close"],
            name="Candlestick",
        )
    )
    # Low minimas
    fig.add_trace(
        go.Scatter(
            x=low_minima.index,
            y=low_minima["Low"],
            mode="markers",
            marker=dict(color="red", size=10),
            name="Low Minima",
        )
    )
    # High maximas
    fig.add_trace(
        go.Scatter(
            x=high_maxima.index,
            y=high_maxima["High"],
            mode="markers",
            marker=dict(color="green", size=10),
            name="High Maxima",
        )
    )
    fig.update_layout(
        title="Candlestick Chart with Extrema",
        xaxis_title="Date",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
    )
    fig.show()


# Example usage with candlestick plot
plot_candlestick_with_extrema(data, low_minima, high_maxima)

In [408]:
# we will search for the best prominence coefficient (at least verify for a few cases):
# BLK: min 08-20, max 08-22 (between aug 13, sep 3incl.)
# agco: min 08-14, max 08-22 (between jul 31, sep 2incl.)
# CPA: min  aug  21aug28(between aug 12, sep 10 incl.)
